In [422]:
import json 
import xmltodict 
import os
import time
import datetime
from tqdm import tqdm
from bs4 import BeautifulSoup as bs
import re
from pymystem3 import Mystem
import pymorphy2
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer as rt
from bson.son import SON
import pprint

import pymongo
from pymongo import TEXT
from bson import ObjectId

# Предобработка данных

## Создание списка с XML

In [423]:
path_60 = '/Users/tonleon/Documents/GitHub/Elena_Shvarts/TEI/60'

In [424]:
def create_list_of_xml_from_dir(path): #функция, которая наши xml-файлы из директории читает и кладет в список
    shvarts_xml= []
    for i, file in enumerate(os.listdir(path)):
        if file.endswith ('.xml'):
            fullname = os.path.join(path, file)
            with open(fullname, encoding='utf8') as xml_file:
                xml = xml_file.read()
                xml = re.sub('/text>\n</TEI>', '/text>\n<ID>'+str(i)+'</ID><file>'+file+'</file></TEI>\n', xml)
                xml = re.sub('<fileDesc>\n', '<fileDesc>\n<root></root>', xml)
                shvarts_xml.append(xml)
                xml_file.close() 
        
    return shvarts_xml

In [425]:
#применяем нашу функцию по доставанию XML к директории, в которой лежат тексты 60-х годов
shvarts_60_xml = create_list_of_xml_from_dir(path_60)

In [426]:
shvarts_60_xml

['<?xml version="1.0" encoding="UTF-8"?>\n<TEI>\n    \n    <teiHeader>\n        <fileDesc>\n<root></root>            <titleStmt>\n                <title>Баллада, которую в конце схватывает паралич</title>\n                <author>Елена Шварц </author>\n            </titleStmt>\n            <respStmt>\n                <resp>Редакторы:</resp>\n                <name>П. Успенский</name> и <name>А. Шеля</name>\n            </respStmt>\n            <respStmt>\n                <resp>Издатели:</resp>\n                <name>Л. Родичева</name> и <name>Е. Сенаторова</name>\n            </respStmt>\n            <sourceDesc>\n                <bibl>\n                    <publisher>Сочинения, том 1. С. 42-43.\n                        <date type="publishing">2013</date></publisher>\n                    <date type="written" when="1969">1969</date>\n                </bibl>\n            </sourceDesc>\n        </fileDesc>\n    </teiHeader>\n    <text>\n        <lg type="poem">\n            <head>Баллада, 

## Подключение к базе данных

In [427]:
'''Пока что начинаем с того, что дропаем нашу базу данных'''
client = pymongo.MongoClient('localhost', 27017) #подключаемся к MongoDB
db = client['admin'] #создаем курсор для конкретной базы/контейнера
db
db['Shvarts_60'].drop()#поскольку мы постоянно пробуем и терять нам нечего, дропаем базу прям сразу же
db['Shvarts_70'].drop()

In [428]:
collection_60 = db["Shvarts_60"] #создаем внутри базы коллекции, где у нас лежат тексты/леммы/токены/строки за 60-е годы
collection_70 = db.Shvarts_70
zeltet = db.Zelenaya_tetrad

In [429]:
example = "<publisher>Сочинения, том 5. С.233 <date type='publishing'>2013</date></publisher>"
example = bs(example, 'xml')
example = example.find('publisher').next_element
# example = example.next_element
print(example)

Сочинения, том 5. С.233 


##   Получаем данные по xml в виде json/словаря (как в тетрадке), а заодно кладем все в базу (операция Create).

In [430]:
'''Создаем класс, как в примере с N+1 и все кладем в него, 
на выходе у нас данные не только в виде строк, но и в виде чисел'''

class Shvarts_poems: 
    def __init__(self):
        self.title=""
        self.root=""
        self.author=""
        self.editors=""
        self.publishers=""
        self.edition=""
        self.date_published=""
        self.date_written=""
        self.text=""
        self.ID=""
        self.file=""
        self.root=""

def getPoemInfoShvarts(text):
    text = re.sub('publishing', 'published', text)
    text = re.sub('Издатели:', '', text)
    text = re.sub('Редакторы:', '', text)
    soup = bs(text, 'xml')
    
    poem=Shvarts_poems()
    
    poem.title = soup.find('title').text
    poem.author = soup.find('author').text
    
    
    poem.editors = ''
    poem.publishers = ''
    for i, resp in enumerate(soup.find_all('respStmt')):
        if i == 0:
            poem.editors += resp.text
        if i == 1:
            poem.publishers += resp.text       
    poem.editors = re.sub('\s+', ' ', poem.editors)
    poem.publishers = re.sub('\s+', ' ', poem.publishers)
    
    
    poem.edition = soup.find('publisher').next_element
    poem.edition = re.sub('\s+', ' ', poem.edition)
    
    
    poem.date_published = soup.find('date', {'type':'published'}).text
    if poem.date_published is not None:
        poem.date_published = int(poem.date_published)
     
    
    poem.date_written = soup.find('date', {'type':'written'}).text
    if poem.date_written in ('', ' '):
        poem.date_written = None
    elif poem.date_written is not None:
        poem.date_written = int(poem.date_written)
        
    poem.ID = soup.find('ID').text
    if poem.ID is not None:
        poem.ID = int(poem.ID)
        
    poem.file = soup.find('file').text
    
    
    #Вот тут новый кусок - мы чекаем, является ли текст дубликатом или нет
    counter = 0
    for character in poem.file:
        if character.isdigit():
            counter+=1
            
    if counter > 4:
        poem.root = 'duplicate'
    else:
        poem.root = None

    poem.text = soup.find('lg').text
    
    return poem

In [431]:
for text in shvarts_60_xml:
    soup = bs(text, 'xml')
    print(soup)

<?xml version="1.0" encoding="utf-8"?>
<TEI>
<teiHeader>
<fileDesc>
<root/> <titleStmt>
<title>Баллада, которую в конце схватывает паралич</title>
<author>Елена Шварц </author>
</titleStmt>
<respStmt>
<resp>Редакторы:</resp>
<name>П. Успенский</name> и <name>А. Шеля</name>
</respStmt>
<respStmt>
<resp>Издатели:</resp>
<name>Л. Родичева</name> и <name>Е. Сенаторова</name>
</respStmt>
<sourceDesc>
<bibl>
<publisher>Сочинения, том 1. С. 42-43.
                        <date type="publishing">2013</date></publisher>
<date type="written" when="1969">1969</date>
</bibl>
</sourceDesc>
</fileDesc>
</teiHeader>
<text>
<lg type="poem">
<head>Баллада, которую в конце схватывает паралич</head>
<lg/>
<l n="1">Поет кобзарь на рынке,</l>
<l n="2">На чем-то небрежно играя,</l>
<l n="3">Скрипят его ботинки,</l>
<l n="4">Как двери рая.</l>
<l n="5">Стоять уставши, пристав сел —</l>
<l n="6">Совсем сегодня нету дел,</l>
<l n="7">И светлыми глазами людоеда,</l>
<l n="8">Казалось, он искал себе обеда.</l>
<

In [432]:
'''также мы пробовали pymystem3, однако у него струкутра не такая удобная и тэги не такие понятные и лаконичные'''
morph=pymorphy2.MorphAnalyzer() 

In [433]:
def putShvartsPoemsInMongo(text):
    tokenizer = rt('\w+')
    # Загружаем текст стихотворения и прочие элементы
    poem = getPoemInfoShvarts(text) 
    
    full_text = {"meta": {"title": poem.title, "author": poem.author, 
                           "editors": poem.editors, "publishers":poem.publishers,
                           "edition":poem.edition, "date_published":poem.date_published,
                           "date_written":poem.date_written, "file":poem.file}, 
                "ID":poem.ID, "root":poem.root, "title": poem.title, "poem_text": poem.text}
    
    text_id = collection_60.insert_one(full_text).inserted_id
    
#     lines = re.split("\n", poem.text)
#     line_num = 1
#     for line in lines:
#         # Токенизируем
#         tokens = tokenizer.tokenize(line)
#         posit = 1
        
#         for token in tokens:
#             info_about_tokens = morph.parse(token)
            
#             # Проверяем, есть ли подобная лемма в нашей базе
#             if lemmas.count_documents({"iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS}) == 0:
#                 lemma_id = lemmas.insert_one({"iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS, 
#                                               "freq": 1, "docs":[text_id]}).inserted_id
#             else:
#                 inis = lemmas.find({"iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS})
#                 lemma_id = inis[0]["_id"]
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$inc": {"freq":1}})
#                 lemmas.find_one_and_update({"_id": lemma_id}, {"$addToSet": {"docs": text_id}}) 
            
#             # Проверяем, есть ли подобный токен в нашей базе
#             if dictionary.count_documents({"token": token, "iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS}) == 0:
#                 token_id = dictionary.insert_one({"token":token, "iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS, "freq": 1, "docs": [text_id]}).inserted_id
#             else:
#                 wrdf = dictionary.find({"token": token, "iniForm": info_about_tokens[0].normal_form, "POS": info_about_tokens[0].tag.POS})
#                 token_id=wrdf[0]["_id"]
#                 dictionary.find_one_and_update({"_id": token_id}, {"$inc":  {"freq": 1}}) 
#                 dictionary.find_one_and_update({"_id": token_id}, {"$addToSet": {"docs": text_id}}) 
  
#             # Добавляем номер строки, идентификатор токена из словаря, сам токен (почему-то не кладется), позицию слова, из какого оно текста.
#             poem_lines.insert_one({"sent_id": line_num, "wordFormId":token_id, "token":token, "position": posit, "textId": text_id})
#             posit += 1  
#         line_num += 1       

In [434]:
'''На этом моменте мы кладем в нашу бд все, что у нас имеется'''
db['Shvarts_60'].drop()
for text in shvarts_60_xml:
    putShvartsPoemsInMongo(text)

collection_60.count_documents({})

36

In [435]:
collection_60.create_index([('poem_text', 'text')], default_language='russian')

'poem_text_text'

## Операция Read - читаем, что у нас в базе данных существует

In [34]:
def find_text(where, which):
    for text in collection_60.find({where:which}, projection={"_id":False, "poem_text": False}):
        print(text)

In [414]:
find_text('root','duplicate')

{'meta': {'title': 'Баллада, которую в конце схватывает паралич', 'author': 'Елена Шварц ', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition': 'Сочинения, том 1. С.42-43. ', 'date_published': 2013, 'date_written': 1969, 'file': '1969_Ballada_kotoruyu_v_kontse_shvatyvaet_paralich_1.xml'}, 'ID': 0, 'root': 'duplicate', 'title': 'Баллада, которую в конце схватывает паралич'}
{'meta': {'title': 'Баллада, которую в конце схватывает паралич', 'author': 'Елена Шварц ', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition': 'Эхо. Париж. № 2. С.92, б/д. ', 'date_published': 1978, 'date_written': 1969, 'file': '1969_Ballada_kotoruyu_v_kontse_shvatyvaet_paralich_2.xml'}, 'ID': 4, 'root': 'duplicate', 'title': 'Баллада, которую в конце схватывает паралич'}
{'meta': {'title': 'Новорожденному', 'author': 'Елена Шварц', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition'

In [420]:
duplicates = []
for text in collection_60.find({'root': None}, projection={"_id":False, "poem_text": False}):
    duplicates.append(text)
print(len(duplicates))

28


## Операция Update - можем обновить наши данные

In [17]:
'''Например мы видим, что в некоторых местах у нас указание даты написания равно None, а мы уже обрели знание,
что текст был написан тогда-то'''

collection_60.find_one_and_update({"meta.title":"Об изобретении паровой машины", "meta.date_written":'1967'},{"$set":{"meta.date_written":1967}}, upsert=False)

In [18]:
for text in collection_60.find({'meta.title':'Об изобретении паровой машины'}):
    print(text)

{'_id': ObjectId('6033e68e77f3c4dcf4f66d51'), 'meta': {'title': 'Об изобретении паровой машины', 'author': 'Елена Шварц ', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition': 'Сочинения, том 5. C.253-254 2013', 'date_published': 2013, 'date_written': None, 'file': '1967_Ob_izobretenii_parovoy_machiny.xml'}, 'ID': 3, 'title': 'Об изобретении паровой машины', 'poem_text': '\nОб изобретении паровой машины\nБыл лекарь механизмов\n Механик Джемс Уатт —\n Подслеповатой линзы\n Он самый нежный брат.\n Совсем он мало спал,\n И при свече до часу\n Он раны бинтовал\n Угрюмому компасу.\n «Ты должен, как распятый,\n Свои раскинуть руки\n И чуять, как собака,\n Тяжёлый запах юга».\n Но в сад и стриженый и бледный,\n Который он любил,\n Однажды вышел он и медный\n Будильник отложил.\n И говорил он, сострадая:\n «Понятно мне, за что из рая\n Нас выгнали. И терпим и молчим.\n Но вот за что железо\n и камень с ним?»\n И в тишине заговорил с ним кто-то —\n Сн

In [37]:
def update_data(collection, title, name, where, item, for_what):
    collection.find_one_and_update({title:name,
                                       where:item},
                                      {"$set":{where:for_what}}, upsert=False)
    
    for text in collection.find({title:name}):
        print(text)
    
update_data(collection_60, 'meta.title', 'Об изобретении паровой машины',
           'meta.date_written', None, 1967)

{'_id': ObjectId('60252569051cf8449e53c682'), 'meta': {'title': 'Об изобретении паровой машины', 'author': 'Елена Шварц ', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition': 'Сочинения, том 5. C.253-254 2013', 'date_published': 2013, 'date_written': 1967, 'file': '1967_Ob_izobretenii_parovoy_machiny.xml'}, 'ID': 3, 'title': 'Об изобретении паровой машины', 'poem_text': '\nОб изобретении паровой машины\nБыл лекарь механизмов\n Механик Джемс Уатт —\n Подслеповатой линзы\n Он самый нежный брат.\n Совсем он мало спал,\n И при свече до часу\n Он раны бинтовал\n Угрюмому компасу.\n «Ты должен, как распятый,\n Свои раскинуть руки\n И чуять, как собака,\n Тяжёлый запах юга».\n Но в сад и стриженый и бледный,\n Который он любил,\n Однажды вышел он и медный\n Будильник отложил.\n И говорил он, сострадая:\n «Понятно мне, за что из рая\n Нас выгнали. И терпим и молчим.\n Но вот за что железо\n и камень с ним?»\n И в тишине заговорил с ним кто-то —\n Сн

## Операция Delete

In [38]:
'''Помним, что там где-то у нас завалялась Агния Львовна - давайте удалим ее, она ж не Шварц'''
print(collection_60.count_documents({}))

for text in collection_60.find({'TEI.teiHeader.title':'Мишка'}):
    print(text)

collection_60.delete_many({'TEI.teiHeader.title':'Мишка'})        

print(collection_60.count_documents({}))

33
33


## Sort, Regex, Aggregation, полнотекстовый поиск

* Sort

In [263]:
for text in collection_60.find({'meta.date_written':{'$gte':1960}}, projection = {'title' : True, 'meta.date_written': True, "_id":False}).sort('meta.date_written'):
    print(text)
# for text in dictionary.find({"freq":{"$gt": 28}}, {"token": True, "freq":True, "_id":False}).sort("freq"):
#     print(text)

{'meta': {'date_written': 1962}, 'title': 'Юродивый'}
{'meta': {'date_written': 1962}, 'title': 'В соборе'}
{'meta': {'date_written': 1963}, 'title': '*** («О ангелы, вы хилы...»)'}
{'meta': {'date_written': 1963}, 'title': 'Свеча Пушкина'}
{'meta': {'date_written': 1963}, 'title': 'Сад'}
{'meta': {'date_written': 1963}, 'title': 'Монолог лодки'}
{'meta': {'date_written': 1964}, 'title': 'Болярыня Морозова'}
{'meta': {'date_written': 1964}, 'title': 'Начало весны'}
{'meta': {'date_written': 1964}, 'title': 'Моим стихам'}
{'meta': {'date_written': 1964}, 'title': 'Море'}
{'meta': {'date_written': 1964}, 'title': 'Повесть'}
{'meta': {'date_written': 1964}, 'title': 'Рыбаки'}
{'meta': {'date_written': 1965}, 'title': '*** («Когда корабль тёк на север...»)'}
{'meta': {'date_written': 1965}, 'title': 'Болезнь'}
{'meta': {'date_written': 1965}, 'title': 'Из зимних стихов'}
{'meta': {'date_written': 1965}, 'title': 'Зимний лес'}
{'meta': {'date_written': 1965}, 'title': 'Из морских стихов'}
{

* RegExp

In [40]:
def search_re(where, word):
    for text in collection_60.find({where:{'$regex':'.*'+word+'.*'}}, projection = {'_id':False, 'meta.title':True, 'poem_text':True}):
        print(text)    

In [41]:
search_re('poem_text','дерев')

{'meta': {'title': 'Из зимних стихов'}, 'poem_text': '\nИз зимних стихов\n\nСегодня подобна зима подушке лебяжьего пуха.\n Перья взбились, облака по бокам небес неслись.\n Снег сыпался так мелко и так сухо,\n как будто целый день просеивали рис.\n И солнце розовое дышит,\n дыханьем так окружено,\n как люди, кони, деревья, крыши\n и голубое моё окно.\n Деревья светятся — и странно\n на землю божий свет вернулся.\n И снова возят землю в телеге деревянной\n лебедь важная и сироты-гуси.\n'}
{'meta': {'title': 'Начало весны'}, 'poem_text': '\nНачало весны\n\nУ дома нашего сегодня дверь болит.\n Он плывёт по весне, суровый и утлый.\n А снег ещё хрустит, как будто бы\n костями\n людоеды хрустят\n в мусорной яме.\n Смотрит в небо стадо луж чёрно-голубых\n тихими глазами сирот и немых.\nНаточила луна из лучей\n сотни синих ножей.\n О, недаром к убийству тянет\n и деревья в схватках дрожат —\n вот и год уже клонит к апрелю,\n вот и звери все уже пришли,\n и даже птицы прилетели,\n что улетали от

* Aggregation

In [279]:
titles_pipeline = [{"$unwind": "$title"},
            {"$group": {"_id": "$title", "count": {"$sum": 1}}},
            {"$sort": SON([("count", -1), ("_id", -1)])}
           ]
pprint = (list(collection_60.aggregate(titles_pipeline)))
pprint

[{'_id': 'Старость княгини Дашковой', 'count': 4},
 {'_id': 'Баллада, которую в конце схватывает паралич', 'count': 3},
 {'_id': 'Рондо с примесью патриотизма', 'count': 2},
 {'_id': 'Новорожденному', 'count': 2},
 {'_id': 'Баллада о спиритическом сеансе и тени Александра Пушкина',
  'count': 2},
 {'_id': 'Юродивый', 'count': 1},
 {'_id': 'Свеча Пушкина', 'count': 1},
 {'_id': 'Сад', 'count': 1},
 {'_id': 'Рыбаки', 'count': 1},
 {'_id': 'Псков', 'count': 1},
 {'_id': 'Похороны', 'count': 1},
 {'_id': 'Повесть', 'count': 1},
 {'_id': 'Об изобретении паровой машины', 'count': 1},
 {'_id': 'Начало весны', 'count': 1},
 {'_id': 'Море', 'count': 1},
 {'_id': 'Монолог лодки', 'count': 1},
 {'_id': 'Моим стихам', 'count': 1},
 {'_id': 'Из морских стихов', 'count': 1},
 {'_id': 'Из зимних стихов', 'count': 1},
 {'_id': 'Зимний лес', 'count': 1},
 {'_id': 'В соборе', 'count': 1},
 {'_id': 'Болярыня Морозова', 'count': 1},
 {'_id': 'Болезнь', 'count': 1},
 {'_id': '*** («Так иногда линяет змей..

In [421]:
titles_pipeline_1 = [{"$unwind": "$title"},
            {"$group": {"_id": "$title"}}
           ]
pprint = (list(collection_60.aggregate(titles_pipeline_1)))

titles = [title for title in collection_60.find({"title": {"$exists": True}}).sort('meta.date_written')]
titles = [title for title in collection_60.find({"root": None}).sort('meta.date_written')]
titles

[{'_id': ObjectId('60351d6177f3c4dcf4f670de'),
  'meta': {'title': 'Юродивый',
   'author': 'Елена Шварц',
   'editors': ' П. Успенский и А. Шеля ',
   'publishers': ' Л. Родичева и Е. Сенаторова ',
   'edition': 'Сочинения, том 5. С.224. ',
   'date_published': 2013,
   'date_written': 1962,
   'file': '1962_Yurodivy.xml'},
  'ID': 1,
  'root': None,
  'title': 'Юродивый',
  'poem_text': '\nЮродивый\n\nГлаза за ниточки дёргая,\nвизжу и в обмороки падаю.\nЯ — юродивый. В снег башкой,\nпоматывая прядями.\nПо снегу, по зелёному\nколешками, колешками.\nПо вам, тупые, взглядами —\nсучочками, полешками.\nЭй, осторожные да аккуратные,\nсолнце падает!\nКак мухи в муссе, туда и обратно,\n в солнце — падалью.\n Дом стоит тортом —\n расступитесь,\n бейте,\n идите к чёрту!\n У меня бог есть,\n у вас — нету.\n Солнце упадёт,\n всегда будет лето.\n\nАлакадзе\n'},
 {'_id': ObjectId('60351d6177f3c4dcf4f670fb'),
  'meta': {'title': 'В соборе',
   'author': 'Елена Шварц, 1948-2010',
   'editors': ' П. 

In [43]:
date_pipeline = [{"$unwind": "$meta.date_published"},
            {"$group": {"_id": "$meta.date_published", "count": {"$sum": 1}}},
            {"$sort": SON([("count", -1), ("_id", -1)])}
           ]
pprint.pprint(list(collection_60.aggregate(date_pipeline)))

[{'_id': 2013, 'count': 30},
 {'_id': 1999, 'count': 1},
 {'_id': 1990, 'count': 1},
 {'_id': 1973, 'count': 1}]


* Full-text search

In [45]:
for text in collection_60.find({"$text": {"$search": 'болотами'}}, projection={'_id':False}):
    print(text)

{'meta': {'title': 'Моим стихам', 'author': 'Елена Шварц, 1948-2010', 'editors': ' П. Успенский и А. Шеля ', 'publishers': ' Л. Родичева и Е. Сенаторова ', 'edition': 'Сочинения, том 5. C.239 2013', 'date_published': 2013, 'date_written': 1964, 'file': '1964_Moim_Stiham.xml'}, 'ID': 23, 'title': 'Моим стихам', 'poem_text': '\nМоим стихам/head>\n            \nВаша крёстная мать — Луна,\n вы одни у меня на свете.\n Как живётся вам без меня,\n мои смертные дети?\n\n\nЛишь волки, к земле припадая,\n в глухой и чёрный, звериный час,\n и ведьмы, над болотом летая,\n быть может, вспомнят ненароком вас.\n\n\n'}


## Проверка кода для бэкэнда

In [46]:
poem_texts = [text for text in collection_60.find({"poem_text": {"$exists": True}})]
for text in poem_text:

SyntaxError: unexpected EOF while parsing (<ipython-input-46-6320861f43cd>, line 2)

In [47]:
def find_text():
    client = pymongo.MongoClient('mongodb://localhost:27017')
    db = client['admin']
    collection_60 = db.Shvarts_60
    texts = [text for text in collection_60.find({"poem_text": {"$exists": True}}, 
                                                 projection={"_id":False, "meta": False})]
    return texts

In [48]:
find_text()

[{'ID': 0,
  'title': 'Юродивый',
  'poem_text': '\nЮродивый\n\nГлаза за ниточки дёргая,\nвизжу и в обмороки падаю.\nЯ — юродивый. В снег башкой,\nпоматывая прядями.\nПо снегу, по зелёному\nколешками, колешками.\nПо вам, тупые, взглядами —\nсучочками, полешками.\nЭй, осторожные да аккуратные,\nсолнце падает!\nКак мухи в муссе, туда и обратно,\n в солнце — падалью.\n Дом стоит тортом —\n расступитесь,\n бейте,\n идите к чёрту!\n У меня бог есть,\n у вас — нету.\n Солнце упадёт,\n всегда будет лето.\n\nАлакадзе\n'},
 {'ID': 1,
  'title': '"душ бестелесных много на земле..."',
  'poem_text': '\n***\nДуш бестелесных много на земле. \n и не заметит сам, как оживёт и засмеётся.\n Как червь в цветке, душа в нём заведётся.\n И так скворцы в скворешник влетают, озираясь,\n так в новый дом жильцы въезжают не спеша,\n так в глиняного человека, неловко улыбаясь,\n вошла когда-то нищая душа.\n Весенний мёртвый снег тихонько позови,\n с песком смешай и с глиной.\n О снежной бабе скажешь: «Оживи!» —\

In [49]:
def search(word):
    texts = [text for text in collection_60.find({"$text": {"$search": word}}, 
             projection={'_id':False, 'meta':False})]

#     poems = []

#     for text in texts:
#         lst = []
#         lst.append(text.get('ID'))
#         lst.append(text.get('title'))
#         poems.append(lst)

    return texts

In [50]:
search('дерево')

[{'ID': 16,
  'title': 'Из зимних стихов',
  'poem_text': '\nИз зимних стихов\n\nСегодня подобна зима подушке лебяжьего пуха.\n Перья взбились, облака по бокам небес неслись.\n Снег сыпался так мелко и так сухо,\n как будто целый день просеивали рис.\n И солнце розовое дышит,\n дыханьем так окружено,\n как люди, кони, деревья, крыши\n и голубое моё окно.\n Деревья светятся — и странно\n на землю божий свет вернулся.\n И снова возят землю в телеге деревянной\n лебедь важная и сироты-гуси.\n'},
 {'ID': 25,
  'title': 'Сад',
  'poem_text': '\nСад\nЕ. Кумпан\n\nПлескался за оградой сад\n и затопил бы город.\n Но цепкие пальцы оград\n его держали за ворот.\n Вошла. Повисли ветви надо мной и смыли,\n ослепили, умерли глаза от горьких, от зелёных брызг.\n Глазам моим куста хватило,\n чтоб нализаться вдрызг.\n Меня ударило об дно огромного сухого сада.\n Умолкли листья, стали глуше.\n Пятипалые корни хватали ограду.\n Дышала трава. И было ей душно.\n Я шла в кустах, как в облаках,\n как чрез г

In [266]:
poem_texts = [text for text in collection_60.find({"poem_text": {"$exists": True}}).sort('meta.date_written')]

In [267]:
poem_texts

[{'_id': ObjectId('603505eb77f3c4dcf4f66ebe'),
  'meta': {'title': 'Юродивый',
   'author': 'Елена Шварц',
   'editors': ' П. Успенский и А. Шеля ',
   'publishers': ' Л. Родичева и Е. Сенаторова ',
   'edition': 'Сочинения, том 5. С.224. ',
   'date_published': 2013,
   'date_written': 1962,
   'file': '1962_Yurodivy.xml'},
  'ID': 1,
  'title': 'Юродивый',
  'poem_text': '\nЮродивый\n\nГлаза за ниточки дёргая,\nвизжу и в обмороки падаю.\nЯ — юродивый. В снег башкой,\nпоматывая прядями.\nПо снегу, по зелёному\nколешками, колешками.\nПо вам, тупые, взглядами —\nсучочками, полешками.\nЭй, осторожные да аккуратные,\nсолнце падает!\nКак мухи в муссе, туда и обратно,\n в солнце — падалью.\n Дом стоит тортом —\n расступитесь,\n бейте,\n идите к чёрту!\n У меня бог есть,\n у вас — нету.\n Солнце упадёт,\n всегда будет лето.\n\nАлакадзе\n'},
 {'_id': ObjectId('603505eb77f3c4dcf4f66edb'),
  'meta': {'title': 'В соборе',
   'author': 'Елена Шварц, 1948-2010',
   'editors': ' П. Успенский и А. Ш

In [268]:
titles = [title for title in collection_60.find({"title": {"$exists": True}}).sort('meta.date_written')]
titles

[{'_id': ObjectId('603505eb77f3c4dcf4f66ebe'),
  'meta': {'title': 'Юродивый',
   'author': 'Елена Шварц',
   'editors': ' П. Успенский и А. Шеля ',
   'publishers': ' Л. Родичева и Е. Сенаторова ',
   'edition': 'Сочинения, том 5. С.224. ',
   'date_published': 2013,
   'date_written': 1962,
   'file': '1962_Yurodivy.xml'},
  'ID': 1,
  'title': 'Юродивый',
  'poem_text': '\nЮродивый\n\nГлаза за ниточки дёргая,\nвизжу и в обмороки падаю.\nЯ — юродивый. В снег башкой,\nпоматывая прядями.\nПо снегу, по зелёному\nколешками, колешками.\nПо вам, тупые, взглядами —\nсучочками, полешками.\nЭй, осторожные да аккуратные,\nсолнце падает!\nКак мухи в муссе, туда и обратно,\n в солнце — падалью.\n Дом стоит тортом —\n расступитесь,\n бейте,\n идите к чёрту!\n У меня бог есть,\n у вас — нету.\n Солнце упадёт,\n всегда будет лето.\n\nАлакадзе\n'},
 {'_id': ObjectId('603505eb77f3c4dcf4f66edb'),
  'meta': {'title': 'В соборе',
   'author': 'Елена Шварц, 1948-2010',
   'editors': ' П. Успенский и А. Ш